# Data augmentation in NLP

In [1]:
text = """Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine."""

# 1 Lexical substitution
## 1.1 Thesaurus-based
### 1.1.1 Dictionary of synonyms

In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
thesaurus = pd.read_csv("thesaurus_synonymes.csv")

In [4]:
# to reduce noise (e.g. without this, "ampere" replacing "a" appears in all words containing an "a")
thesaurus = thesaurus[thesaurus['Noms'].apply(lambda x: len(x) >= 3)] 

In [5]:
%%time

word_to_change = random.randint(0, len(text.replace(",","").split())-1)
while text.replace(",","").split()[word_to_change] not in list(thesaurus.Noms):
    word_to_change = random.randint(0, len(text.replace(",","").split())-1)

# original text
print(text)

# new text
print(" "+text.replace(text.replace(",","").split()[word_to_change]+" " , " "+random.choice(thesaurus[thesaurus.Noms == text.replace(",","").split()[word_to_change]].Synonymes.iloc[0].split('|')[1:])+" "))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
 Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 5.98 ms


In [6]:
def thesaurus_replacement(text, num):    
    list_word_to_change = []
    for word in range(num):
        word_to_change = random.randint(0, len(text.replace(",","").split())-1)
        while text.replace(",","").split()[word_to_change] not in list(thesaurus.Noms):
            word_to_change = random.randint(0, len(text.replace(",","").split())-1)
        list_word_to_change.append(word_to_change)
            
    for word_to_change in list_word_to_change:
        text = text.replace(" "+text.replace(",","").split()[word_to_change]+" " , " "+random.choice(thesaurus[thesaurus.Noms == text.replace(",","").split()[word_to_change]].Synonymes.iloc[0].split('|')[1:])+" ")

    return text

In [7]:
%%time
# original text
print(text)

# new text
print(thesaurus_replacement(text,3))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un concours ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 12.1 ms


### 1.1.2 WordNet Thesaurus but in French (reuse in part 4.6)

In [8]:
import nltk
# nltk.download('omw-1.4')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [9]:
def get_synonyms(word):
    synonyms = set()
    
    for syn in wordnet.synsets(word, lang='fra'): 
        for lem in syn.lemmas('fra'): 
            synonym = lem.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' azertyuiopqsdfghjklmwxcvbn'])
            synonyms.add(synonym) 
    
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)


def wordnet_replacement(words, num):
    words = words.split()
    stop_words = set(stopwords.words('french'))
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= num: # only replace up to n words
            break

    sentence = ' '.join(new_words)

    return sentence

In [10]:
%%time
# original text
print(text)

# new text
print(wordnet_replacement(text,3))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un effort ce vendredi vingt et un avril en battant le Chinois Fan Zhendong, proportion 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 1.5 s


### 1.1.3 WoNef Thesaurus

In [11]:
wonef = pd.read_csv("Wonef.csv")

In [12]:
# to reduce noise
wonef = wonef[wonef['word'].apply(lambda x: len(str(x)) >= 3)] 

In [13]:
%%time
word_to_change = random.randint(0, len(text.replace(",","").split())-1)
while text.replace(",","").split()[word_to_change] not in list(wonef.word):
    word_to_change = random.randint(0, len(text.replace(",","").split())-1)

# original text
print(text)

# new text
print(text.replace(text.replace(",","").split()[word_to_change], random.choice(wonef[wonef.word == text.replace(",","").split()[word_to_change]].synonyms.iloc[0].split('|'))))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, nombre 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 9.98 ms


In [14]:
def wonef_replacement(text, num):    
    list_word_to_change = []
    for word in range(num):
        word_to_change = random.randint(0, len(text.replace(",","").split())-1)
        while text.replace(",","").split()[word_to_change] not in list(wonef.word):
            word_to_change = random.randint(0, len(text.replace(",","").split())-1)
        list_word_to_change.append(word_to_change)
            
    for word_to_change in list_word_to_change:
        text = text.replace(" "+text.replace(",","").split()[word_to_change] , " "+random.choice(wonef[wonef.word == text.replace(",","").split()[word_to_change]].synonyms.iloc[0].split('|')))
    return text

In [15]:
%%time
# original text
print(text)

# new text
print(wonef_replacement(text,3))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un réussite ce vendredi 21 avril en battant le Chinois Fan Zhendong, finalité 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 54.7 ms


## 1.2 Based on word embedding

### 1.2.1 Fauconnier

In [16]:
from gensim.models import KeyedVectors

The 20 models of https://fauconnier.github.io/#data :  
12 trained on frWaC (https://wacky.sslmit.unibo.it/doku.php?id=corpora): a 1.6 billion word corpus built from the Web by limiting the exploration to the .fr domain and using medium frequency words from the Le Monde Diplomatique corpus and core French vocabulary lists as seeds. The dataset dates from 2008.
- "https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_200_cbow_cut0.bin" # (2.7Gb)  
- "https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin" # (120Mb)  
- "https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_200_skip_cut100.bin" # (120Mb)  
- "https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_500_skip_cut100.bin" # (298Mb)  
- "https://embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_500_skip_cut200.bin" # (202Mb)  
- "https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin" # (229Mb)  
- "https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_skip_cut100.bin" # (229Mb)  
- "https://embeddings.net/embeddings/frWac_no_postag_no_phrase_700_skip_cut50.bin" # (494Mb)  
- "https://embeddings.net/embeddings/frWac_postag_no_phrase_700_skip_cut50.bin" # (577Mb)  
- "https://embeddings.net/embeddings/frWac_postag_no_phrase_1000_skip_cut100.bin" # (520Mb)  
- "https://embeddings.net/embeddings/frWac_no_postag_phrase_500_cbow_cut10.bin" # (2Gb)  
- "https://embeddings.net/embeddings/frWac_no_postag_phrase_500_cbow_cut100.bin" # (289Mb)  

8 trained on FrWiki dump (https://dumps.wikimedia.org/frwiki/frwiki/), 600 million words. The dataset is from 2010.
- "https://embeddings.net/embeddings/frWiki_no_lem_no_postag_no_phrase_1000_cbow_cut100.bin" # (253Mb)  
- "https://embeddings.net/embeddings/frWiki_no_lem_no_postag_no_phrase_1000_cbow_cut200.bin" # (195Mb)  
- "https://embeddings.net/embeddings/frWiki_no_lem_no_postag_no_phrase_1000_skip_cut100.bin" # (253Mb)  
- "https://embeddings.net/embeddings/frWiki_no_lem_no_postag_no_phrase_1000_skip_cut200.bin" # (195Mb)  
- "https://embeddings.net/embeddings/frWiki_no_phrase_no_postag_500_cbow_cut10.bin" # (128Mb)  
- "https://embeddings.net/embeddings/frWiki_no_phrase_no_postag_700_cbow_cut100.bin" # (106Mb)  
- "https://embeddings.net/embeddings/frWiki_no_phrase_no_postag_1000_skip_cut100.bin" # (151Mb)  
- "https://embeddings.net/embeddings/frWiki_no_phrase_no_postag_1000_skip_cut200.bin" # (121Mb)  


We can also mention:
- The 2 (a general, a politician) proposed by a team of Polytechnique: http://nlp.polytechnique.fr/word2vec#french trained on 33Go of texts. Seems to date from 2021 (= the most recent). The resources are available for academic/non profit use by sending a request by email to mvazirg~lix.polytechnique.fr.
- A trained model on Wikipedia 2018 lemmatized : https://zenodo.org/record/3241447#.ZD_iTs7P1PY

In [17]:
model = KeyedVectors.load_word2vec_format("frWac_non_lem_no_postag_no_phrase_500_skip_cut200.bin", binary=True, unicode_errors="ignore")
words = model.vocab.keys()

In [18]:
%%time

word_to_change = random.randint(0, len(text.replace(",","").split())-1)

while text.replace(",","").split()[word_to_change] not in words:
    word_to_change = random.randint(0, len(text.replace(",","").split())-1)

# original text
print(text)

# new text
print(text.replace(text.replace(",","").split()[word_to_change],model.most_similar(text.replace(",","").split()[word_to_change])[0][0]))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vdedredi 21 avril de battant le Chinois Fan Zhdedong, numéro 1 mondial, de quarts de finale du tournoi WTT Champions de Macao, de Chine.
Wall time: 290 ms


In [19]:
def word2vec_replacement(text, num, model_name):
    model = KeyedVectors.load_word2vec_format(model_name, binary=True, unicode_errors="ignore")
    words = model.vocab.keys()
    
    list_word_to_change = []
    for word in range(num):
        index_to_change = random.randint(0, len(text.replace(",","").split())-1)
        # maybe add an additional condition verifying that the n randomly chosen numbers are all different
        while text.replace(",","").split()[index_to_change] not in words:
            index_to_change = random.randint(0, len(text.replace(",","").split())-1)
        list_word_to_change.append(index_to_change)
            
    for word_to_change in list_word_to_change:
        text = text.replace(text.replace(",","").split()[word_to_change],model.most_similar(text.replace(",","").split()[word_to_change])[0][0])

    return text

In [20]:
%%time
# original text
print(text)

# new text
print(word2vec_replacement(text, 3, "frWac_non_lem_no_postag_no_phrase_500_skip_cut200.bin"))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un vainqueur ce vdedredi 21 avril de battant le Chinois Fan Zhdedong, numéro 1 mondiaux, de quarts de finale du tournoi WTT Champions de Macao, de Chine.
Wall time: 925 ms


### 1.2.2 FastText

In [21]:
# version with fasttext lib

In [22]:
import fasttext

In [23]:
# Models can be downloaded manually from https://fasttext.cc/docs/en/crawl-vectors.html
model = fasttext.load_model('cc.fr.300.bin') # 7Go (much heavier than a transformer or another word2vec)

# # We can also do
# from huggingface_hub import hf_hub_download
# model = hf_hub_download(repo_id="facebook/fasttext-fr-nearest-neighbors", filename="model.bin")

In [24]:
%%time
word_to_change = random.randint(0, len(text.replace(",","").split())-1)

# original text
print(text)

# new text
print(text.replace(text.split()[word_to_change],model.get_nearest_neighbors(text.split()[word_to_change])[0][1]))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale le tournoi WTT Champions de Macao, en Chine.
Wall time: 15.2 s


In [25]:
def fastext_replacement(text, num):
    
    list_word_to_change = []
    for word in range(num):
        list_word_to_change.append(random.randint(0, len(text.replace(",","").split())-1)) 
        # maybe add an additional condition verifying that the n randomly chosen numbers are all different
        
    for word_to_change in list_word_to_change:
        text = text.replace(text.split()[word_to_change],model.get_nearest_neighbors(text.split()[word_to_change])[0][1])

    return text

In [26]:
%%time
# original text
print(text)

# new text
print(fastext_replacement(text,3))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 22 ans, a réalisé un exploit ce vEndredi 21 avril En battant le Chinois Fan ZhEndong numéro 1 mondial, En quarts de finale du tournoi WTT Champions de Macao, En Chine.
Wall time: 1.84 s


## 1.3 Based on a masked language model (BERT)

In [27]:
from transformers import pipeline

In [29]:
camembert_fill_mask = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base", top_k=7)

In [30]:
# By modifying a random word

In [31]:
%%time

word_to_change = random.randint(0, len(text.replace(",","").split())-1)
new_text = []
for word in range(len(text.split())):
    if word == word_to_change :
        new_text.append(text.split()[word].replace(text.split()[word_to_change],'<mask>'))
    else :
        new_text.append(text.split()[word])
results = camembert_fill_mask(' '.join(new_text))

# original text
print(text)

# new text
if all(isinstance(elem, list) for elem in results):
    # Sometimes the output from the pipeline is a single list containing the requested top_k and sometimes the pipeline returns a list of lists.
    # So we must handle the two different cases.
    if text.replace("\n","") == results[0][0]['sequence']: # make sure that the new word chosen by CamemBERT is different from the one in the original text
        print(results[0][random.randint(1,len(results)-1)]['sequence'])
    else:
        print(results[0][0]['sequence'])
else :
    if text.replace("\n","") == results[0]['sequence']: # idem
        print(results[random.randint(1,len(results)-1)]['sequence'])
    else:
        print(results[0]['sequence'])

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce dimanche 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 170 ms


### TODO
- Version where only specific words are modified via POS
- Version where we can change several words and not only one  
Indeed, the `fill-mask` pipeline only works for inputs with exactly one token masked. HF added an experimental option to support multiple masks.  
The returned values are raw model output, and correspond to disjoint probabilities where one might expect joint probabilities.  
See https://github.com/huggingface/transformers/pull/10222

## 1.4 Based on a TF-IDF
TODO

## 2. Back-translation

### 2.1. Marian (Helsinki-NLP models)
Full doc : https://huggingface.co/transformers/model_doc/marian.html 

In [32]:
from transformers import pipeline

In [33]:
%%time
fwd_scr = 'fr'  # source language
fwd_trg = 'en'  # target language
rev_scr = 'en'  # source language
rev_trg = 'fr'  # target language

forward_translator = pipeline("translation", f'Helsinki-NLP/opus-mt-{fwd_scr}-{fwd_trg}', f'Helsinki-NLP/opus-mt-{fwd_scr}-{fwd_trg}')
reverse_translator = pipeline("translation", f'Helsinki-NLP/opus-mt-{rev_scr}-{rev_trg}', f'Helsinki-NLP/opus-mt-{rev_scr}-{rev_trg}')
new_text = reverse_translator(forward_translator(text)[0]['translation_text'])

# original text
print(text)

# new text
print(new_text[0]['translation_text'])

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a fait un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, leader mondial, en quart de finale du tournoi WTT Champions à Macao, en Chine.
Wall time: 7.98 s


In [34]:
def backtranslation(text, scr_lang, trg_lang):
    forward_translator = pipeline("translation", f'Helsinki-NLP/opus-mt-{scr_lang}-{trg_lang}', f'Helsinki-NLP/opus-mt-{scr_lang}-{trg_lang}')
    reverse_translator = pipeline("translation", f'Helsinki-NLP/opus-mt-{trg_lang}-{scr_lang}', f'Helsinki-NLP/opus-mt-{trg_lang}-{scr_lang}')
    new_text = reverse_translator(forward_translator(text)[0]['translation_text'])
    return new_text[0]['translation_text']

In [35]:
%%time

# original text
print(text)

# new text
print(backtranslation(text, "fr","en"))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a fait un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, leader mondial, en quart de finale du tournoi WTT Champions à Macao, en Chine.
Wall time: 8.15 s


### 2.2.2 M2M100
Full doc : https://huggingface.co/transformers/model_doc/m2m_100.html  
Sumamry : a single model with about 400M parameters (proposed by Facebook) that manages about 100 languages  

Two implementations are available: one based on the HF pipeline function, the other using the m2m100 model card code.  
The pipeline seems to take much longer to run. So it may not be the best approach to take.

In [36]:
%%time

forward_translator = pipeline('translation', 'facebook/m2m100_418M', src_lang=fwd_scr, tgt_lang=fwd_trg)
reverse_translator = pipeline('translation', 'facebook/m2m100_418M', src_lang=rev_scr, tgt_lang=rev_trg)
new_text = reverse_translator(forward_translator(text)[0]['translation_text'])

# original text
print(text)

# new text
print(new_text[0]['translation_text'])

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, âgé de 19 ans, a fait un résultat vendredi 21 avril en défaisant le chinois Fan Zhendong, numéro un mondial, dans les quarts de finale du Tournoi des champions du WTT à Macao, en Chine.
Wall time: 48.2 s


In [37]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [38]:
%%time
# forward_translator
tokenizer.src_lang = "fr"
encoded = tokenizer(text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True
                       
# forward_translator
src_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
tokenizer.src_lang = "en"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("fr"))

# original text
print(text)

# new text
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

C:\Users\lbourdois\Anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, âgé de 19 ans, a fait un résultat vendredi 21 avril en défaisant le chinois Fan Zhendong, numéro un mondial, dans les quarts de finale du Tournoi des champions du WTT à Macao, en Chine.
Wall time: 24.4 s


# 3. Transformation of the text surface
=> is not used in French!

# 4. Random noise injection

## 4.1 Spelling mistakes injection
TODO :
- check if there is a difference between https://raw.githubusercontent.com/ybisk/charNMT-noise/master/noise/fr.natural and WiCopaCO
- add the errors listed by Wikipedia fr

In [39]:
import re
import pandas as pd
df_wicopaco = pd.read_csv("df_wicopaco.csv")
len(df_wicopaco)

29979

In [40]:
%%time

# spelling mistakes injection
text_test = text + " Il sera décoré pour ce succès."
for i in range(len(df_wicopaco)):
    new_text =  re.sub(" "+df_wicopaco.correction[i]+" "," "+df_wicopaco.error[i]+" ", text_test) 
    # add a regex to take into account the word containing an error + the punctuation
new_text

Wall time: 1.84 s


'Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine. Il sera décroré pour ce succès.'

In [41]:
%%time

# spelling mistakes correction
text_test = text + " Il sera décroré pour ce succès."
for i in range(len(df_wicopaco)):
    new_text =  re.sub(" "+df_wicopaco.error[i]+" "," "+df_wicopaco.correction[i]+" ", text_test) 
    # add a regex to take into account the word containing an error + the punctuation
new_text

Wall time: 1.8 s


'Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine. Il sera décoré pour ce succès.'

## 4.2 Typing errors injection
https://github.com/makcedward/nlpaug/blob/7c748d98b25cb008165e40b5955d193cbd3a26d6/nlpaug/res/char/keyboard/fr.json (serves as a basis but should be completed to take into account keyboards with numeric keypads + cell phone keyboards) 

In [42]:
import json

with open('keyboard_fr.txt') as file:
    data = file.read()
keyboard_fr = json.loads(data)

In [43]:
def keyboard_replacement(text, n):
    
    list_word_to_change = []
    for word in range(n):
        list_word_to_change.append(random.randint(0, len(text.replace(",","").split())-1)) 
        # maybe add an additional condition verifying that the n randomly chosen numbers are all different
        
    for word_to_change in list_word_to_change:
        letter_to_change = random.choice(text.split()[word_to_change]).lower()
        while letter_to_change not in keyboard_fr.keys():
            letter_to_change = random.choice(text.split()[word_to_change]).lower()
        text = text.replace(text.split()[word_to_change],text.split()[word_to_change].lower().replace(letter_to_change,random.choice(keyboard_fr[letter_to_change])))
    return text

In [44]:
# original text
print(text)

# new text
print(keyboard_replacement(text,3))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avrip en battant le Chinois can Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de m&c&o, en Chine.


## 4.3 Unigram noise injection
TODO

## 4.4 Noise injection
TODO

## 4.5 Mixed sentences
TODO

## 4.6 Random insertion

In [45]:
def random_insertion(words, n):
    words = words.split()
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

In [46]:
# original text
print(text)

# new text
print(random_insertion(text,3))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, liste des codes aita des aroportsa a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro seigneur 1 mondial, en quarts  de finale du tournoi WTT Champions de Macao, en Chine.


## 4.7 Random swap

In [47]:
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

def random_swap(words, n):
    words = words.split()
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    sentence = ' '.join(new_words)
    return sentence

In [48]:
# original text
print(text)

# new text
print(random_swap(text,len(text)//50))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le Fan français Chine. Lebrun, 19 ans, a réalisé Macao, exploit un vendredi 21 avril en battant le Chinois pongiste Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de ce en Alexis


## 4.8 Random deletion

In [49]:
def random_deletion(words, p):
    words = words.split()
    # if there's only one word, don't delete it
    if len(words) == 1:
        return words

    # randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    # if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]
    sentence = ' '.join(new_words)
    return sentence

In [50]:
# original text
print(text)

# new text
print(random_deletion(text,0.05))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant Chinois Fan Zhendong, 1 mondial, quarts de finale du tournoi WTT Champions Macao, Chine.


# 5. Cross-over augmentation

# 6. Manipulating the syntax tree

In [51]:
from inflecteur import inflecteur # nice lib that should be completed and optimized
inflecteur = inflecteur()
inflecteur.load_dict()

Loading	 dela-fr-public...
Done.


## 6.1 Time manipulation

In [52]:
# original text
print(text)

# new text
print(inflecteur.inflect_sentence(text, tense='Futur').replace(" \\' ","'"))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.


C:\Users\lbourdois\Anaconda3\lib\site-packages\transformers\pipelines\token_classification.py:135: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Le pongiste français Alexis Lebrun, 19 ans, aura réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine .


## 6.2 Gender manipulation

In [53]:
# original text
print(text)

# new text
print(inflecteur.inflect_sentence(text, gender='f').replace(" \\' ","'"))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
L pongiste française Alexis Lebrun, 19 ans, a réalisé une exploit cette vendredi 21 avril en battant l Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine .


## 6.3 Number manipulation

In [54]:
# original text
print(text)

# new text
print(inflecteur.inflect_sentence(text, number='p').replace(" \\' ","'"))

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Les pongistes français Alexis Lebrun, 19 ans, ont réalisé des exploits ces vendredis 21 avrils en battant les Chinois Fan Zhendong, numéros 1 mondial, en quarts de finales du tournois WTT Champions de Macao, en Chines .


# 7. MixUp
## 7.1 Word Mix Up

## 7.2 Sentence Mix Up

# 8. Generative methods
## 8.1 Generate paraphrases 

The version of FrenchT0 available on HF is bad in this task. See if it is possible to get a version of the model at a more advanced time or else train a model to generate paraphrases.

# 9. Text simplification
## 9.1 Text summary

In [55]:
from transformers import pipeline
summarizer = pipeline("summarization","moussaKam/barthez-orangesum-abstract","moussaKam/barthez-orangesum-abstract")
# https://huggingface.co/csebuetnlp/mT5_multilingual_XLSum can be interesting for multilingual

In [56]:
# original text
print(text)

# new text
print(summarizer(text, min_length=len(text)//10, max_length=len(text)//2)[0]['summary_text'])

Your max_length is set to 102, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit en battant le Chinois Fan Zhendong, numéro 1 mondial.


## 9.2 Simplification
The version of FrenchT0 available on HF is bad in this task. See if it is possible to get a version of the model at a more advanced time or else train a model to generate paraphrases.